In [5]:
#bibliotecas básicas para manipulção de dado.
import pandas as pd
import numpy as np

Realizo a leitura dos registros dos alunos evadidos da Escola de Minas. Dados retirados da UFOP, link no final do prgrama.

In [6]:
evadidos_1 = pd.read_csv('graduacao_evadidos.csv',sep=';', encoding='latin-1')
evadidos_2 = pd.read_csv('evadidos19_1.csv',sep=';', encoding='latin-1')
evadidos_3 = pd.read_csv('evadidos18_2.csv',sep=';', encoding='latin-1')
evadidos_4 = pd.read_csv('evadidos18_1.csv',sep=';', encoding='latin-1')

Tratamento dos bancos de dado:

In [7]:
#Unir os ultimos 3 periodos em um dataframe, todos continham cabeçalho identico.
evadidos_EM = pd.concat([evadidos_2, evadidos_3, evadidos_4], ignore_index=False, sort = False)

#Transformo a cabeçalho de todas as colunas em letra maiuscula e sem acento.
evadidos_1.columns = evadidos_1.columns.str.upper()
evadidos_EM.columns = evadidos_EM.columns.str.upper()
evadidos_EM = evadidos_EM.rename(columns={"UNIDADE ACADÊMICA": "UNIDADE_ACADEMICA","TIPO EVASÃO": "TIPO_EVASAO","ALUNOS": "EVADIDOS"})

#Junto o dataframe com todos os evadidos da EM
evadidos = pd.concat([evadidos_EM, evadidos_1], ignore_index=False, sort = False)

#Seleciono a Escola de Minas e limpo a coluna "CURSO"
evadidos = evadidos.loc[evadidos['UNIDADE_ACADEMICA'] == 'ESCOLA DE MINAS']
evadidos[['Resto','CURSO']] = (evadidos.CURSO.astype(str).str.split('-', expand = True))
evadidos = evadidos.drop(columns =["Resto"]) 
evadidos['TIPO_EVASAO'] = evadidos['TIPO_EVASAO'].replace('REOPÇÃO', 'REOPCAO')
evadidos['TIPO_EVASAO'] = evadidos.TIPO_EVASAO.str.replace('POR ', '')
evadidos.tail()

,ANO,SEMESTRE,CAMPUS,UNIDADE_ACADEMICA,CURSO,MODALIDADE,FORMATO,TIPO_EVASAO,EVADIDOS
3113,2017,2,OURO PRETO,ESCOLA DE MINAS,MEC,BACHARELADO,PRESENCIAL,DESLIGAMENTO,3
3114,2017,2,OURO PRETO,ESCOLA DE MINAS,MEC,BACHARELADO,PRESENCIAL,JUBILAMENTO,2
3115,2017,2,OURO PRETO,ESCOLA DE MINAS,MET,BACHARELADO,PRESENCIAL,CANCELAMENTO,32
3116,2017,2,OURO PRETO,ESCOLA DE MINAS,MET,BACHARELADO,PRESENCIAL,DESLIGAMENTO,9
3117,2017,2,OURO PRETO,ESCOLA DE MINAS,MET,BACHARELADO,PRESENCIAL,JUBILAMENTO,1


<h2> Observando a evasão de cada curso temos: <h2>

Vamos gerar as tabelas a com a taxa de evasão para o motivo de cada curso da Escola de Minas, com elas é possivel realizar análises estatísticas relevantes no R.

Precisamos agrupar a quatidade de evadidos em relação ao curso, ano e motivo da evasão. 

In [8]:
#Tabela base para a formulação dos gráficos e análises.
evadidos_ano_curso_motivo = evadidos.groupby(['ANO', 'CURSO', 'TIPO_EVASAO']).EVADIDOS.sum().to_frame(name = 'Evadidos').reset_index()
evadidos_ano_curso_motivo = evadidos_ano_curso_motivo.loc[evadidos_ano_curso_motivo['CURSO'] == 'PRO']
evadidos_ano_curso_motivo.tail()

,ANO,CURSO,TIPO_EVASAO,Evadidos
460,2018,PRO,NRM,9
487,2019,PRO,CANCELAMENTO,5
488,2019,PRO,DESLIGAMENTO,1
489,2019,PRO,DESLIGAMENTO PRAZO MÁXIMO,4
490,2019,PRO,NRM,1


<h4>Problemas:</h4>

- Gerar os valores nulos nos Evadidos quando o tipo de evasão não tiver ocorrido no ano.

- Adicionar na coluna "motivos" o total de evadidos em todas as modalidades em cada ano. 

- Adicionar o número de matriculados de cada curso na data correta.

In [9]:
tipos_evasao = pd.DataFrame(evadidos['TIPO_EVASAO'].unique())
tipos_evasao = tipos_evasao.rename(columns={list(tipos_evasao)[0]: 'Tipo de evasao'})

cursos = pd.DataFrame(evadidos['CURSO'].unique())
cursos = cursos.rename(columns={list(cursos)[0]: 'CURSOS'})

ANOS = pd.DataFrame(evadidos['ANO'].unique())
ANOS = ANOS.rename(columns={list(ANOS)[0]: 'ANO'})

todos_motivos = pd.Series(['TODOS SOMADOS'])
todos_motivos = pd.DataFrame(todos_motivos)
todos_motivos = todos_motivos.rename(columns={list(todos_motivos)[0]: 'Tipo de evasao'})

tipos_evasao = pd.concat([tipos_evasao, todos_motivos], ignore_index=True)

In [21]:
import itertools
um = pd.DataFrame(list(itertools.product(cursos['CURSOS'],tipos_evasao['Tipo de evasao'])), columns=['curso', 'motivo'])
dois = pd.DataFrame(list(itertools.product(cursos['CURSOS'],ANOS['ANO'])), columns=['curso', 'ano'])
base = um.merge(dois, left_on = 'curso', right_on = 'curso')

base = base.loc[base['curso'] == 'PRO']
base.tail()

AMB    17
URB    17
CIV    17
MIN    17
GEO    17
AUT    17
MET    17
PRO    17
ARQ    17
MEC    17
Name: curso, dtype: int64

In [12]:
#obtendo as evasões totais e filtrando o curso de PRO
evadidos_totais = evadidos.groupby(['ANO', 'CURSO']).EVADIDOS.sum().to_frame(name = 'Evadidos').reset_index()
evadidos_totais['motivo'] = 'TODOS SOMADOS'
evadidos_totais = evadidos_totais.loc[evadidos_totais['CURSO'] == 'PRO']
evadidos_totais.tail()
# #Termino de criar meu dataframe que me possibilitará adicionar as evasões totais e os anos sem evasão.
# base = base.merge(evadidos_totais,how='left', left_on = ['ano','curso','motivo'], right_on = ['ANO', 'CURSO','motivo'])
# base = base.drop(columns= ['ANO','CURSO'])
# base = base.fillna(0)

# base.tail()

,ANO,CURSO,Evadidos,motivo
104,2015,PRO,32,TODOS SOMADOS
113,2016,PRO,33,TODOS SOMADOS
122,2017,PRO,29,TODOS SOMADOS
131,2018,PRO,29,TODOS SOMADOS
141,2019,PRO,11,TODOS SOMADOS


Ainda nos resta agregar o total de matriculados em cada ano e curso.

In [145]:
#Matriculados na Escola de Minas
EM_total = pd.read_csv('emtotalordenado.csv',sep=';', encoding='latin-1')
EM_total[['resto','Curso']] = EM_total['Curso'].str.split('-', expand=True)
EM_total = EM_total.drop(columns= ['resto'])

#para agrupar pela data transformei os anos em variavel tipo data e somei os periodos.
EM_total['SemestreLetivo'] = pd.to_datetime(EM_total['SemestreLetivo'], format='%Y/%m')
#EM_total_pro = EM_total.loc[EM_total['Curso'] == 'PRO']

matriculados_cursos = (pd.DataFrame(EM_total_pro.groupby(EM_total_pro['SemestreLetivo'].dt.year).TotalGeral.sum())).reset_index()
matriculados_cursos.tail()

,SemestreLetivo,TotalGeral
12,2015,762
13,2016,757
14,2017,713
15,2018,712
16,2019,364


In [146]:
#Criação da tabela com Taxa de Evasão
tabela_motivos = tabela_motivos.merge(matriculados_cursos,how='left', left_on=['ano'], right_on= ['SemestreLetivo'])
tabela_motivos = tabela_motivos.drop(columns= ['SemestreLetivo'])
tabela_motivos['Evasoes a cada 100 alunos'] = round((tabela_motivos['Evadidos']/tabela_motivos['TotalGeral'])*100,2)
tabela_motivos.tail()

KeyError: 'ano'

In [147]:
tabela_motivos.to_csv('tabela_cursos_motivo_ano.csv')

<h2>Vamos repetir o processo<h2>

In [148]:
motivos_EM = evadidos.groupby(['ANO', 'TIPO_EVASAO']).EVADIDOS.sum().to_frame(name = 'Evadidos').reset_index()
tree = pd.DataFrame(list(itertools.product(tipos_evasao['Tipo de evasao'],ANOS['ANO'])), columns=['TIPO_EVASAO', 'ANO'])

tree.tail()

,TIPO_EVASAO,ANO
148,TODOS SOMADOS,2013
149,TODOS SOMADOS,2014
150,TODOS SOMADOS,2015
151,TODOS SOMADOS,2016
152,TODOS SOMADOS,2017


In [149]:
#obtendo as evasões totais
evadidos_totais = evadidos.groupby(['ANO']).EVADIDOS.sum().to_frame(name = 'Evadidos').reset_index()
evadidos_totais['TIPO_EVASAO'] = 'TODOS SOMADOS'
evadidos_totais.tail()

,ANO,Evadidos,TIPO_EVASAO
12,2015,303,TODOS SOMADOS
13,2016,299,TODOS SOMADOS
14,2017,252,TODOS SOMADOS
15,2018,303,TODOS SOMADOS
16,2019,140,TODOS SOMADOS


In [150]:
#Termino de criar meu dataframe que me possibilitará adicionar os evadidos com TODOS SOMADOS e os anos sem evasão.
base = tree.merge(evadidos_totais,how='left', left_on = ['TIPO_EVASAO','ANO'], right_on = ['TIPO_EVASAO', 'ANO'])
base = base.fillna(0)

base.tail()

,TIPO_EVASAO,ANO,Evadidos
148,TODOS SOMADOS,2013,234.0
149,TODOS SOMADOS,2014,240.0
150,TODOS SOMADOS,2015,303.0
151,TODOS SOMADOS,2016,299.0
152,TODOS SOMADOS,2017,252.0


In [151]:
base.columns

Index(['TIPO_EVASAO', 'ANO', 'Evadidos'], dtype='object')

In [152]:
motivos_EM.tail()

,ANO,TIPO_EVASAO,Evadidos
83,2019,DESLIGAMENTO,37
84,2019,DESLIGAMENTO PRAZO MÁXIMO,7
85,2019,NRM,10
86,2019,REOPCAO,2
87,2019,TRANSFERENCIA,4


In [153]:
#merge base com dados
tabela_motivos = motivos_EM.merge(base,how='right', left_on = ['ANO','TIPO_EVASAO'], right_on = ['ANO', 'TIPO_EVASAO'])

#Finalmente consigo identificar todos os anos em que não houveram evasões em cada modalidade.
tabela_motivos = tabela_motivos.fillna(0)

tabela_motivos['Evadidos'] = tabela_motivos['Evadidos_x'] + tabela_motivos['Evadidos_y']
tabela_motivos = tabela_motivos.drop(columns= ['Evadidos_x','Evadidos_y'])
tabela_motivos.tail()

,ANO,TIPO_EVASAO,Evadidos
148,2013,TODOS SOMADOS,234.0
149,2014,TODOS SOMADOS,240.0
150,2015,TODOS SOMADOS,303.0
151,2016,TODOS SOMADOS,299.0
152,2017,TODOS SOMADOS,252.0


In [154]:
#para agrupar pela data transformei os anos em variavel tipo data e somei os periodos.
matriculados_cursos_geral = (pd.DataFrame(EM_total.groupby(EM_total['SemestreLetivo'].dt.year).TotalGeral.sum())).reset_index()
matriculados_cursos.tail()

,SemestreLetivo,TotalGeral
12,2015,762
13,2016,757
14,2017,713
15,2018,712
16,2019,364


In [155]:
#Criação da tabela com Taxa de Evasão
tabela_motivos = tabela_motivos.merge(matriculados_cursos_geral,how='left', left_on=['ANO'], right_on= ['SemestreLetivo'])
#tabela_motivos = tabela_motivos.merge(tree,how='left', left_on=['ANO','TIPO_EVASAO'], right_on= ['ANO','TIPO_EVASAO'])

tabela_motivos = tabela_motivos.drop(columns= ['SemestreLetivo'])
tabela_motivos['Evasoes a cada 100 alunos'] = round((tabela_motivos['Evadidos']/tabela_motivos['TotalGeral'])*100,2)
tabela_motivos.tail()

,ANO,TIPO_EVASAO,Evadidos,TotalGeral,Evasoes a cada 100 alunos
148,2013,TODOS SOMADOS,234.0,5957,3.93
149,2014,TODOS SOMADOS,240.0,6200,3.87
150,2015,TODOS SOMADOS,303.0,6284,4.82
151,2016,TODOS SOMADOS,299.0,6358,4.70
152,2017,TODOS SOMADOS,252.0,6119,4.12


In [156]:
tabela_motivos.tail()

,ANO,TIPO_EVASAO,Evadidos,TotalGeral,Evasoes a cada 100 alunos
148,2013,TODOS SOMADOS,234.0,5957,3.93
149,2014,TODOS SOMADOS,240.0,6200,3.87
150,2015,TODOS SOMADOS,303.0,6284,4.82
151,2016,TODOS SOMADOS,299.0,6358,4.70
152,2017,TODOS SOMADOS,252.0,6119,4.12


In [157]:
tabela_motivos.to_csv('tabela_motivos.csv')